# Structured Data Preprocessing

In [1]:
import pandas as pd
import numpy as np

In [2]:
countries = pd.read_csv("../countries.csv").fillna(0.0)

In [3]:
countries.sample(5)

,Country,Region,Population,Area (sq. mi.),Pop. Density (per sq. mi.),Coastline (coast/area ratio),Net migration,Infant mortality (per 1000 births),GDP ($ per capita),Literacy (%),Phones (per 1000),Arable (%),Crops (%),Other (%),Climate,Birthrate,Deathrate,Agriculture,Industry,Service
125,Malawi,SUB-SAHARAN AFRICA,13013926,118480,"109,8","0,00",0,"103,32",600.0,"62,7","7,9","23,38","1,49","75,13",2,"43,13","19,33","0,342","0,158","0,499"
215,Uruguay,LATIN AMER. & CARIB,3431932,176220,"19,5","0,37","-0,32","11,95",12800.0,"98,0","291,4","7,43","0,23","92,34",3,"13,91","9,05","0,093","0,311","0,596"
210,Uganda,SUB-SAHARAN AFRICA,28195754,236040,"119,5","0,00",0,"67,83",1400.0,"69,9","3,6","25,88","10,65","63,47",2,"47,35","12,24","0,311","0,222","0,469"
206,Turkey,NEAR EAST,70413958,780580,"90,2","0,92",0,"41,04",6700.0,"86,5","269,5","30,93","3,31","65,76",3,"16,62","5,97","0,117","0,298","0,585"
150,Nicaragua,LATIN AMER. & CARIB,5570129,129494,"43,0","0,70","-1,22","29,11",2300.0,"67,5","39,7","15,94","1,94","82,12",2,"24,51","4,45","0,165","0,275","0,56"


In [4]:
countries = countries.rename(columns={
                        "Region":"region",
                          "Country":"country",
                          "Population":"population",
                          "Pop. Density (per sq. mi.)":"density",
                          "Coastline (coast/area ratio)":"coast",
                          "Net migration":"migrate",
                          "Literacy (%)":"literacy",
                          "Area (sq. mi.)":"area",
                          "Infant mortality (per 1000 births)":"inf_live",
                          "GDP ($ per capita)":"gdp",
                          "Phones (per 1000)":"phones","Service":"service",
                          "Arable (%)":"arable","Agriculture":"agriculture","Industry":"industry",
                          "Crops (%)":"crops","Birthrate":"birthrate","Deathrate":"deathrate",
                          "Other (%)":"other",
                          "Climate":"climate",
                         })

all the regions:

In [5]:
countries.sample(3)

,country,region,population,area,density,coast,migrate,inf_live,gdp,literacy,phones,arable,crops,other,climate,birthrate,deathrate,agriculture,industry,service
19,Belgium,WESTERN EUROPE,10379067,30528,"340,0","0,22","1,23","4,68",29100.0,"98,0","462,6","23,28","0,4","76,32",3,"10,38","10,27","0,01","0,24","0,749"
117,Liberia,SUB-SAHARAN AFRICA,3042004,111370,"27,3","0,52",0,"128,87",1000.0,"57,5","2,3","3,95","2,28","93,77",2,"44,77","23,1","0,769","0,054","0,177"
108,Kiribati,OCEANIA,105432,811,"130,0","140,94",0,"48,52",800.0,0,"42,7","2,74","50,68","46,58",2,"30,65","8,26","0,089","0,242","0,668"


In [6]:
countries.columns

Index(['country', 'region', 'population', 'area', 'density', 'coast',
       'migrate', 'inf_live', 'gdp', 'literacy', 'phones', 'arable', 'crops',
       'other', 'climate', 'birthrate', 'deathrate', 'agriculture', 'industry',
       'service'],
      dtype='object')

In [7]:
for c in ["density","migrate","coast","phones",'other', 'climate', 'birthrate', 'deathrate', 'agriculture', 'industry',
       'service']:
    countries[c] = countries[c].apply(lambda x:float(str(x).replace(",",".")))

### Data Frame Preprocessing management

In [126]:
import os
class col_core:
    def __init__(self,col_name,save_dir = ".matchbox/fields",debug=False):
        os.system("mkdir -p %s"%(save_dir))
        self.col_name = col_name
        self.debug = debug
        self.save_dir = save_dir
        if self.save_dir[-1]!="/": self.save_dir+="/"
            
        self.meta = dict()
    
    def save_meta(self):
        np.save(self.save_dir+str(self.col_name)+".npy",self.meta)
        
    def set_meta(self,meta=None):
        """
        pass meta dict values to obj attributes
        """
        if meta == None: meta = self.meta
        for k,v in meta.items():
            if self.debug: print("setting:\t%s\t%s"%(k,v))
            setattr(self,k,v)
        
    def load_meta(self,path=None):
        if path==None:
            path = self.save_dir+str(self.col_name)+".npy"
        self.meta = np.load(path).tolist()
        self.set_meta(self.meta)
        
        if self.meta["coltype"] == "tabulate": self.make_sub() # make sub-objects out of meta
        
    def make_meta(self):
        for attr in self.make_meta_list:
            self.meta[attr]  = getattr(self,attr)
            
    def check_dim(self,data):
        return pd.DataFrame(data,columns=self.dim_names)
    
class categorical(col_core):
    def __init__(self,col_name,save_dir = ".matchbox/fields"):
        super(categorical,self).__init__(col_name, save_dir)
        self.coltype = "categorical"
        self.make_meta_list = ["col_name","coltype","idx2cate","cate2idx","width","eye","dim_names"]
        
    def build(self,pandas_s,max_ = 20):
        assert max_>1, "max should be bigger than 1"
        
        vcount = pd.DataFrame(pandas_s.value_counts())
        
        print(vcount)
        
        self.cate_full = list(vcount.index.tolist())
        self.cate_list = self.cate_full[:max_-1]
        
        # build dictionary
        self.idx2cate = dict((k,v) for k,v in enumerate(self.cate_list))
        self.idx2cate.update({len(self.cate_list):"_other"})
        
        self.cate2idx = dict((v,k) for k,v in self.idx2cate.items())
        self.eye = np.eye(len(self.cate2idx))
        
        self.width = len(self.cate2idx)
        
        self.dim_names = list("%s -> %s"%(self.col_name,k) for k in self.cate2idx.keys())
        self.make_meta()
        
    def trans2idx(self,cate):
        """Translate category to index
        """
        try:
            return self.cate2idx[cate]
        except:
            return self.cate2idx["_other"]
        
    def prepro_idx(self,pandas_s):
        return pandas_s.apply(self.trans2idx)
    
    def prepro(self,pandas_s):
        return self.eye[self.prepro_idx(pandas_s).values]
    
class categorical_idx(col_core):
    def __init__(self,col_name,save_dir = ".matchbox/fields"):
        super(categorical_idx,self).__init__(col_name, save_dir)
        self.coltype = "categorical_idx"
        self.dim_names = [self.col_name]
        self.width = 1
        self.make_meta_list = ["col_name","coltype","idx2cate","cate2idx","width","dim_names"]
        
    def build(self,pandas_s,max_ = 20):
        assert max_>1, "max should be bigger than 1"
        
        vcount = pd.DataFrame(pandas_s.value_counts())
        
        print(vcount)
        
        self.cate_full = list(vcount.index.tolist())
        self.cate_list = self.cate_full[:max_-1]
        
        # build dictionary
        self.idx2cate = dict((k,v) for k,v in enumerate(self.cate_list))
        self.idx2cate.update({len(self.cate_list):"_other"})
        
        self.cate2idx = dict((v,k) for k,v in self.idx2cate.items())
        
        self.make_meta()
        
    def trans2idx(self,cate):
        try:
            return self.cate2idx[cate]
        except:
            return self.cate2idx["_other"]
        
    def prepro(self,pandas_s, expand=True):
        x = pandas_s.apply(self.trans2idx).values
        if expand:x = np.expand_dims(x,-1)
        return x
    
class minmax(col_core):
    def __init__(self,col_name,fillna=0.0,save_dir = ".matchbox/fields"):
        """minmax scaler: scale to 0~1"""
        super(minmax,self).__init__(col_name, save_dir)
        self.coltype = "minmax"
        self.fillna = fillna
        self.dim_names = [self.col_name]
        self.width = 1
        self.make_meta_list = ["col_name","coltype","min_","max_","range","width","dim_names"]
        
    def build(self,pandas_s=None,min_=None,max_=None):
        if type(pandas_s) != pd.core.series.Series:
            assert (min_!=None) and (max_!=None), "If no pandas series is set you have to set min_,max_ value"
            self.min_ = min_
            self.max_ = max_
            
        else:
            pandas_s = pandas_s.fillna(self.fillna)
            if min_ == None:
                self.min_ = pandas_s.min()
            else:
                self.min_ = min_
            if max_ == None:
                self.max_ = pandas_s.max()
            else:
                self.max_ = max_
                
        self.range = self.max_-self.min_
        assert self.range!=0, "the value range is 0"
        print("min_:%.3f \tmax_:%.3f\t range:%.3f"%(self.min_,self.max_,self.range))
        self.make_meta()
        
    def prepro(self,data,expand=True):
        x = (np.clip(data.values,self.min_,self.max_)-self.min_)/self.range
        if expand:x = np.expand_dims(x,-1)
        return x
        
class tabulate(col_core):
    def __init__(self,table_name,save_dir = ".matchbox/fields"):
        super(tabulate,self).__init__(table_name, save_dir)
        self.coltype = "tabulate"
        self.cols=dict()
        
        self.save_dir = save_dir
        if self.save_dir[-1] != "/":
            self.save_dir = "%s/"%(self.save_dir)
        
        self.make_meta_list = ["col_name","coltype","cols","dim_names"]
        
    def build_url(self,metalist):
        for url in metalist:
            meta_dict = np.load(url).tolist()
            self.cols[meta_dict["col_name"]] = meta_dict
        self.make_dim()
        self.make_meta()
        
    def build(self,*args):
        for obj in args:
            self.cols[obj.col_name] = obj.meta
        self.make_sub()
        self.make_dim()
        self.make_meta()
            
    def make_col(self,meta):
        """
        creat sub obj according to sub meta
        """
        col_name = meta["col_name"]
        
        setattr(self,col_name,eval(meta["coltype"])(col_name))
        getattr(self,col_name).set_meta(meta)
        if meta["coltype"] == "tabulate": 
            getattr(self,col_name).make_sub()
            getattr(self,col_name).meta = meta
        
    def make_sub(self):
        """
        create sub-objects according to meta
        """
        for k,meta in self.cols.items():
            self.make_col(meta)
        
    def make_dim(self):
        self.dim_names = []
        
        for k,meta in self.cols.items():
            for sub_dim in meta["dim_names"]:
                self.dim_names.append("%s -> %s"%(self.col_name, sub_dim))
            
        self.width = len(self.dim_names)
        
    def prepro(self,data):
        """
        data being a pandas dataframe
        """
        data_list = []
        
        for k,v in self.meta["cols"].items():
            # preprocess the data for every column
            col = getattr(self,k)
            if v["coltype"] == "tabulate":
                data_list.append(col.prepro(data))
            else:
                data_list.append(col.prepro(data[k]))
        return np.concatenate(data_list,axis = 1)

### Categorical data test

In [127]:
region =categorical("region")

In [128]:
region.build(countries.region)

                                     region
SUB-SAHARAN AFRICA                       51
LATIN AMER. & CARIB                      45
ASIA (EX. NEAR EAST)                     28
WESTERN EUROPE                           28
OCEANIA                                  21
NEAR EAST                                16
C.W. OF IND. STATES                      12
EASTERN EUROPE                           12
NORTHERN AFRICA                           6
NORTHERN AMERICA                          5
BALTICS                                   3


In [129]:
region.prepro(countries.region)[:5,:]

array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [130]:
region.save_meta()

In [131]:
region.load_meta()

### Minmax test

In [132]:
area =minmax("area")

In [133]:
area.build(countries["area"])
area.save_meta()

min_:2.000 	max_:17075200.000	 range:17075198.000


In [134]:
area =minmax("area")
area.load_meta()

In [135]:
area.prepro(countries["area"])[:20,0]

array([3.79203802e-02, 1.68349439e-03, 1.39485235e-01, 1.15372015e-05,
       2.72910452e-05, 7.30122134e-02, 5.85644746e-06, 2.58269333e-05,
       1.62041342e-01, 1.74510421e-03, 1.11858147e-05, 4.50176215e-01,
       4.91168536e-03, 5.07156637e-03, 8.16271647e-04, 3.88282467e-05,
       8.43316722e-03, 2.51241596e-05, 1.21578678e-02, 1.78773915e-03])

In [136]:
migrate = minmax("migrate")

In [137]:
migrate.build(countries["migrate"])

min_:-20.990 	max_:23.060	 range:44.050


### DataFrame Data Test

In [138]:
tb1 = tabulate("tb1")

In [139]:
tb1.build(area,region,migrate)

In [140]:
sample_x = tb1.prepro(countries)[:3]
sample_x

array([[0.03792038, 0.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 1.        ],
       [0.00168349, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.3645857 ],
       [0.13948523, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 0.        , 0.        , 0.4676504 ]])

In [141]:
tb1.check_dim(sample_x)

,tb1 -> area,tb1 -> region -> SUB-SAHARAN AFRICA,tb1 -> region -> LATIN AMER. & CARIB,tb1 -> region -> ASIA (EX. NEAR EAST),tb1 -> region -> WESTERN EUROPE,tb1 -> region -> OCEANIA,tb1 -> region -> NEAR EAST,tb1 -> region -> C.W. OF IND. STATES,tb1 -> region -> EASTERN EUROPE,tb1 -> region -> NORTHERN AFRICA,tb1 -> region -> NORTHERN AMERICA,tb1 -> region -> BALTICS,tb1 -> region -> _other,tb1 -> migrate
0,0.037920,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000
1,0.001683,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.364586
2,0.139485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.467650


In [142]:
tb1.save_meta()

### Checking load table config

In [143]:
tb_load = tabulate("check_load")

In [144]:
tb_load.load_meta(".matchbox/fields/first.npy")

In [145]:
tb_load.prepro(countries[:3])

array([[0.03792038, 0.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 1.        ],
       [0.00168349, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.3645857 ],
       [0.13948523, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 0.        , 0.        , 0.4676504 ]])

### Table in a table

In [146]:
tb_big = tabulate("big_table")

In [147]:
tb_big.build(region,tb1)

In [150]:
sample_2 = tb_big.prepro(countries[:5])
sample_2

array([[0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        3.79203802e-02, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.68349439e-03, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 3.64585698e-01],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00

In [151]:
tb_big.check_dim(sample_2)

,big_table -> region -> SUB-SAHARAN AFRICA,big_table -> region -> LATIN AMER. & CARIB,big_table -> region -> ASIA (EX. NEAR EAST),big_table -> region -> WESTERN EUROPE,big_table -> region -> OCEANIA,big_table -> region -> NEAR EAST,big_table -> region -> C.W. OF IND. STATES,big_table -> region -> EASTERN EUROPE,big_table -> region -> NORTHERN AFRICA,big_table -> region -> NORTHERN AMERICA,...,big_table -> tb1 -> region -> WESTERN EUROPE,big_table -> tb1 -> region -> OCEANIA,big_table -> tb1 -> region -> NEAR EAST,big_table -> tb1 -> region -> C.W. OF IND. STATES,big_table -> tb1 -> region -> EASTERN EUROPE,big_table -> tb1 -> region -> NORTHERN AFRICA,big_table -> tb1 -> region -> NORTHERN AMERICA,big_table -> tb1 -> region -> BALTICS,big_table -> tb1 -> region -> _other,big_table -> tb1 -> migrate
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.364586
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.467650
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.006356
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.626334


In [152]:
tb_big.save_meta()

In [153]:
tb_big_load = tabulate("tb_big_load")

In [154]:
tb_big_load.load_meta(".matchbox/fields/big_table.npy")

In [156]:
tb_big_load.check_dim(tb_big_load.prepro(countries[:5]))

,big_table -> region -> SUB-SAHARAN AFRICA,big_table -> region -> LATIN AMER. & CARIB,big_table -> region -> ASIA (EX. NEAR EAST),big_table -> region -> WESTERN EUROPE,big_table -> region -> OCEANIA,big_table -> region -> NEAR EAST,big_table -> region -> C.W. OF IND. STATES,big_table -> region -> EASTERN EUROPE,big_table -> region -> NORTHERN AFRICA,big_table -> region -> NORTHERN AMERICA,...,big_table -> tb1 -> region -> WESTERN EUROPE,big_table -> tb1 -> region -> OCEANIA,big_table -> tb1 -> region -> NEAR EAST,big_table -> tb1 -> region -> C.W. OF IND. STATES,big_table -> tb1 -> region -> EASTERN EUROPE,big_table -> tb1 -> region -> NORTHERN AFRICA,big_table -> tb1 -> region -> NORTHERN AMERICA,big_table -> tb1 -> region -> BALTICS,big_table -> tb1 -> region -> _other,big_table -> tb1 -> migrate
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.364586
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.467650
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.006356
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.626334
